In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Hide FutureWarning
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import pyranges as pr

path = '/project/hfa_work/ENCODE/data'
alignment_path = 'alignment_v45'
genome_file = 'GRCh38.p14.genome.fa'
genome_path = os.path.join(path, 'gencode_human/version_45', genome_file)
window_padding = 100

# Import

In [4]:
metadata_file = 'reads/metadata_tissue.tsv'
metadata = pd.read_csv(os.path.join(path, metadata_file), sep='\t')
tissues = metadata['group'].unique()

In [5]:
CAGE_path = '/project/hfa_work/ENCODE/code/snakemake-pipeline/resources/CAGE/'
# filter out tissues with no corresponding CAGE data
tissues = [tissue for tissue in tissues if os.path.exists(os.path.join(CAGE_path, f'{tissue}.bed'))]
tissues

['aorta', 'brain', 'colon', 'heart', 'lung', 'muscle']

In [6]:
# isotools_path = '/project/hfa_work/ENCODE/code/notebooks/results/isotools_'
fofn_path = '/project/hfa_work/ENCODE/code/snakemake-pipeline/results/tissues_gtf.fofn'
# fofn_path = '/project/hfa_work/ENCODE/code/snakemake-pipeline/results/flair/tissue_gtfs.fofn'

tool_gtfs = pd.read_csv(fofn_path, sep='\t', header=None, names=['tool', 'tissue', 'gtf'])
# check that there are no duplicates of tool + tissue
assert tool_gtfs.groupby(['tool', 'tissue']).size().max() == 1
# filter for tissues with CAGE data
tool_gtfs = tool_gtfs[tool_gtfs['tissue'].isin(tissues)]
tool_gtfs

,tool,tissue,gtf
0,flair,aorta,/project/hfa_work/ENCODE/code/snakemake-pipeli...
1,flair,brain,/project/hfa_work/ENCODE/code/snakemake-pipeli...
2,flair,colon,/project/hfa_work/ENCODE/code/snakemake-pipeli...
3,flair,heart,/project/hfa_work/ENCODE/code/snakemake-pipeli...
4,flair,lung,/project/hfa_work/ENCODE/code/snakemake-pipeli...
5,flair,muscle,/project/hfa_work/ENCODE/code/snakemake-pipeli...
6,isotools,aorta,/project/hfa_work/ENCODE/code/snakemake-pipeli...
7,isotools,brain,/project/hfa_work/ENCODE/code/snakemake-pipeli...
8,isotools,colon,/project/hfa_work/ENCODE/code/snakemake-pipeli...
9,isotools,heart,/project/hfa_work/ENCODE/code/snakemake-pipeli...


In [7]:
display_tool = 'flair'
display_tissue = 'brain'

In [8]:
CAGE_data = {}
for tissue in tissues:
    CAGE_data[tissue] = pr.read_bed(os.path.join(CAGE_path, f'{tissue}.bed'))
    CAGE_data[tissue] = CAGE_data[tissue].assign('CAGE_score', lambda x: x['Score']).drop(drop='Score')

tool_gtfs['pr'] = tool_gtfs['gtf'].apply(lambda x: pr.read_gtf(x))

In [9]:
tool_gtfs.loc[(tool_gtfs['tool'] == display_tool) & (tool_gtfs['tissue'] == display_tissue), 'pr'].values[0]

,Chromosome,Source,Feature,Start,End,Score,Strand,Frame,gene_id,transcript_id,exon_number
0,chr1,FLAIR,transcript,175387,175644,.,+,.,chr1:175000,m64109_220203_225821/159646893/ccs,NaN
1,chr1,FLAIR,exon,175387,175644,.,+,.,chr1:175000,m64109_220203_225821/159646893/ccs,0
2,chr1,FLAIR,transcript,197700,199762,.,+,.,chr1:197000,m64109_220205_045735/17237446/ccs,NaN
3,chr1,FLAIR,exon,197700,199762,.,+,.,chr1:197000,m64109_220205_045735/17237446/ccs,0
4,chr1,FLAIR,transcript,265404,267777,.,+,.,chr1:265000,m64109_220205_045735/39125266/ccs,NaN
...,...,...,...,...,...,...,...,...,...,...,...
821086,chrY,FLAIR,exon,307731,307880,.,-,.,ENSG00000292358.1,m64109_220203_225821/137103082/ccs,2
821087,chrY,FLAIR,exon,311418,311627,.,-,.,ENSG00000292358.1,m64109_220203_225821/137103082/ccs,3
821088,chrY,FLAIR,exon,312765,315020,.,-,.,ENSG00000292358.1,m64109_220203_225821/137103082/ccs,4
821089,chrY,FLAIR,exon,316913,317051,.,-,.,ENSG00000292358.1,m64109_220203_225821/137103082/ccs,5


# Find TSS based on strand

In [10]:
# Filter out all positions where extending the window would go out of bounds
tool_gtfs['pr'] = tool_gtfs['pr'].apply(lambda x: x.subset(lambda y: y.Start - window_padding >= 0)) # and y.end + window_padding <= x.chromsizes[0]))
# no chromsizes yet
# tool_gtfs['pr'] = tool_gtfs['pr'].apply(lambda x: x.filter(lambda y: y.start - window_padding >= 0 and y.end + window_padding <= x.chromsizes[0]))

In [11]:
def tss_from_transcriptome(tool, tissue, transcriptome):
    transcriptome = transcriptome[transcriptome.Feature == 'transcript']
    tss_sites = transcriptome.five_end()
    # plus = transcriptome[transcriptome.Strand == '+']
    # plus.End = plus.Start + 1
    # minus = transcriptome[transcriptome.Strand == '-']
    # minus.Start = minus.End - 1
    tss_sites_deduplicated = tss_sites.drop_duplicate_positions()
    tss_beds(tool, tissue, tss_sites_deduplicated)
    print(tool, tissue, tss_sites.df.shape[0] - tss_sites_deduplicated.df.shape[0], "duplicate positions dropped")
    return tss_sites_deduplicated.drop()

def tss_beds(tool, tissue, tss_sites_deduplicated):
    # two beds per transcriptome, one for window inside of transcript and one outside
    # TODO: Does this need a check for the end too? Would require a chromsizes file
    outside = tss_sites_deduplicated.extend({"5": window_padding})
    outside.to_bed(f'/project/hfa_work/ENCODE/code/snakemake-pipeline/results/tss_ratio/tss_outside_{tool}_{tissue}.bed', keep=False)
    inside = tss_sites_deduplicated.extend({"3": window_padding})
    inside.to_bed(f'/project/hfa_work/ENCODE/code/snakemake-pipeline/results/tss_ratio/tss_inside_{tool}_{tissue}.bed', keep=False)

tool_gtfs['tss_sites'] = tool_gtfs.apply(lambda x: tss_from_transcriptome(x['tool'], x['tissue'], x['pr']), axis=1)
tool_gtfs.loc[(tool_gtfs['tool'] == display_tool) & (tool_gtfs['tissue'] == display_tissue), 'tss_sites'].values[0]

flair aorta 19826 duplicate positions dropped
flair brain 32504 duplicate positions dropped
flair colon 21453 duplicate positions dropped
flair heart 95768 duplicate positions dropped
flair lung 50988 duplicate positions dropped
flair muscle 17123 duplicate positions dropped
isotools aorta 3961 duplicate positions dropped
isotools brain 7747 duplicate positions dropped
isotools colon 3814 duplicate positions dropped
isotools heart 32430 duplicate positions dropped
isotools lung 12135 duplicate positions dropped
isotools muscle 3470 duplicate positions dropped


,Chromosome,Start,End,Strand
0,chr1,175387,175388,+
1,chr1,197700,197701,+
2,chr1,265404,265405,+
3,chr1,266854,266855,+
4,chr1,271731,271732,+
...,...,...,...,...
111403,chrY,1392112,1392113,-
111404,chrY,1452928,1452929,-
111405,chrY,2500975,2500976,-
111406,chrY,2500649,2500650,-


# CAGE support

In [12]:
CAGE_data[display_tissue]

,Chromosome,Start,End,Name,Strand,CAGE_score
0,chr1,629179,629180,"chr1:629179..629180,+",+,1
1,chr1,629210,629211,"chr1:629210..629211,+",+,1
2,chr1,629212,629213,"chr1:629212..629213,+",+,1
3,chr1,629620,629621,"chr1:629620..629621,+",+,1
4,chr1,629625,629626,"chr1:629625..629626,+",+,1
...,...,...,...,...,...,...
3059674,chrY,20782681,20782682,"chrY:20782681..20782682,-",-,1
3059675,chrY,21303006,21303007,"chrY:21303006..21303007,-",-,1
3059676,chrY,25626516,25626517,"chrY:25626516..25626517,-",-,1
3059677,chrY,26401653,26401654,"chrY:26401653..26401654,-",-,1


In [13]:
# Extend positions by 20bp in each direction
CAGE_windows = {tissue: CAGE_data[tissue][['CAGE_score']].extend(window_padding) for tissue in tissues}
CAGE_windows[display_tissue]

,Chromosome,Start,End,Strand,CAGE_score
0,chr1,629079,629280,+,1
1,chr1,629110,629311,+,1
2,chr1,629112,629313,+,1
3,chr1,629520,629721,+,1
4,chr1,629525,629726,+,1
...,...,...,...,...,...
3059674,chrY,20782581,20782782,-,1
3059675,chrY,21302906,21303107,-,1
3059676,chrY,25626416,25626617,-,1
3059677,chrY,26401553,26401754,-,1


In [14]:
def add_CAGE_Support(tss_sites, CAGE_windows):
    merged = tss_sites.join(CAGE_windows, strandedness='same', how='left')
    clustered = merged.cluster(slack=-1).boundaries("Cluster", agg={"CAGE_score": "sum"}).drop(drop=["Cluster"])
    return clustered

tool_gtfs['CAGE_clusters'] = tool_gtfs.apply(lambda x: add_CAGE_Support(x['tss_sites'], CAGE_windows[x['tissue']]), axis=1)

In [15]:
tool_gtfs.loc[(tool_gtfs['tool'] == display_tool) & (tool_gtfs['tissue'] == display_tissue), 'CAGE_clusters'].values[0]

,Chromosome,Start,End,Strand,CAGE_score
0,chr1,175387,175388,+,-1
1,chr1,197700,197701,+,-1
2,chr1,265404,265405,+,-1
3,chr1,266854,266855,+,-1
4,chr1,271731,271732,+,-1
...,...,...,...,...,...
111403,chrY,1452928,1452929,-,1
111404,chrY,1596449,1596450,-,-1
111405,chrY,2500649,2500650,-,-1
111406,chrY,2500975,2500976,-,4


# RNA-seq data

In [18]:
path = "/project/hfa_work/ENCODE/code/snakemake-pipeline/results/rnaseq/depth_{side}_{tool}_{tissue}.txt"

In [26]:
def tss_ratio(tool, tissue, tss_sites):
    # reads in output from samtools depth
    # chr, pos, [depth for each sample]
    inside_depth_df = pd.read_csv(path.format(side='inside', tool=tool, tissue=tissue),
                               sep='\t',
                               header=None,
                               names=["Chromosome", "Start"] + [f"inside_sample_{i}" for i in range(len(metadata[metadata.group == tissue]))])
    inside_depth_df['Start'] = inside_depth_df['Start'] - 1
    inside_depth_df['End'] = inside_depth_df['Start'] + 1
    inside_depth = pr.PyRanges(inside_depth_df)
    tss_inside_ranges = tss_sites.assign("TSS", lambda x: x.Start).extend({"3": window_padding})
    tss_inside_joined = tss_inside_ranges.join(inside_depth, strandedness=False, how='left')

    sample_columns_inside = tss_inside_joined.columns[tss_inside_joined.columns.str.startswith('inside_sample_')].values.tolist()
    agg = {col: "mean" for col in sample_columns_inside}
    agg["TSS"] = "first"
    agg["CAGE_score"] = "first"
    tss_inside_joined = tss_inside_joined.cluster(slack=-1).boundaries("Cluster", agg=agg).drop("Cluster")
    tss_inside_joined = tss_inside_joined.assign("Start", lambda x: x.TSS).assign("End", lambda x: x.TSS + 1).drop("TSS")

    outside_depth_df = pd.read_csv(path.format(side='outside', tool=tool, tissue=tissue),
                                 sep='\t',
                                 header=None,
                                 names=["Chromosome", "Start"] + [f"outside_sample_{i}" for i in range(len(metadata[metadata.group == tissue]))])
    outside_depth_df['Start'] = outside_depth_df['Start'] - 1
    outside_depth_df['End'] = outside_depth_df['Start'] + 1
    outside_depth = pr.PyRanges(outside_depth_df)
    tss_outside_ranges = tss_sites.assign("TSS", lambda x: x.Start).extend({"5": window_padding})
    tss_outside_joined = tss_outside_ranges.join(outside_depth, strandedness=False, how='left')

    sample_columns_outside = tss_outside_joined.columns[tss_outside_joined.columns.str.startswith('outside_sample_')].values.tolist()
    agg = {col: "mean" for col in sample_columns_outside}
    agg["TSS"] = "first"
    agg["CAGE_score"] = "first"
    tss_outside_joined = tss_outside_joined.cluster(slack=-1).boundaries("Cluster", agg=agg).drop("Cluster")
    tss_outside_joined = tss_outside_joined.assign("Start", lambda x: x.TSS).assign("End", lambda x: x.TSS + 1).drop("TSS")

    print(f"Imported depth information for {tool} {tissue}. Calculating ratio...")

    combined = tss_inside_joined.join(tss_outside_joined, strandedness=False)
    combined = combined.drop(["Start_b", "End_b", "Strand_b", "CAGE_score_b"])
    combined = combined.apply(lambda x: sample_tss_ratios(tissue, x))
    combined = combined.apply(lambda x: mean_tss_ratio(tissue, x))
    return combined

def sample_tss_ratios(tissue, df):
    for i in range(len(metadata[metadata.group == tissue])):
        df[f"tss_ratio_sample_{i}"] = ((df[f"inside_sample_{i}"] + 0.01) / (df[f"outside_sample_{i}"] + 0.01)).where(df[f"inside_sample_{i}"].gt(0), 0)
        df.drop([f"inside_sample_{i}", f"outside_sample_{i}"], axis=1, inplace=True)
    return df

def mean_tss_ratio(tissue, df):
    num_samples = len(metadata[metadata.group == tissue])
    ratio_columns = [f"tss_ratio_sample_{i}" for i in range(num_samples)]
    df["TSS_ratio"] = df[ratio_columns].mean(axis=1)
    return df

tool_gtfs['CAGE_and_TSS'] = tool_gtfs.apply(lambda x: tss_ratio(x['tool'], x['tissue'], x['CAGE_clusters']), axis=1)

Imported depth information for flair aorta. Calculating ratio...
Imported depth information for flair brain. Calculating ratio...
Imported depth information for flair colon. Calculating ratio...
Imported depth information for flair heart. Calculating ratio...
Imported depth information for flair lung. Calculating ratio...
Imported depth information for flair muscle. Calculating ratio...
Imported depth information for isotools aorta. Calculating ratio...
Imported depth information for isotools brain. Calculating ratio...
Imported depth information for isotools colon. Calculating ratio...
Imported depth information for isotools heart. Calculating ratio...
Imported depth information for isotools lung. Calculating ratio...
Imported depth information for isotools muscle. Calculating ratio...


In [27]:
tool_gtfs.loc[(tool_gtfs['tool'] == display_tool) & (tool_gtfs['tissue'] == display_tissue), 'CAGE_and_TSS'].values[0]

,Chromosome,Start,End,Strand,CAGE_score,tss_ratio_sample_0,tss_ratio_sample_1,tss_ratio_sample_2,tss_ratio_sample_3,TSS_ratio
0,chr1,175387,175388,+,-1,6.039859,6.604772,4.858931,0.0,4.375890
1,chr1,197700,197701,+,-1,0.076653,0.026797,0.374133,0.0,0.119396
2,chr1,265404,265405,+,-1,0.000000,0.000000,0.000000,0.0,0.000000
3,chr1,266854,266855,+,-1,0.631859,16.833974,0.247599,0.0,4.428358
4,chr1,271731,271732,+,-1,0.000000,0.000000,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...
107794,chrY,1452928,1452929,-,1,0.000000,0.000000,0.000000,0.0,0.000000
107795,chrY,1596449,1596450,-,-1,0.000000,0.000000,0.000000,0.0,0.000000
107796,chrY,2500649,2500650,-,-1,0.000000,0.000000,0.000000,0.0,0.000000
107797,chrY,2500975,2500976,-,4,0.000000,0.000000,0.000000,0.0,0.000000


# Summary

In [33]:
tool_gtfs['num_transcripts'] = tool_gtfs['CAGE_and_TSS'].apply(lambda x: x.length)
tool_gtfs['CAGE support absolute'] = tool_gtfs['CAGE_and_TSS'].apply(lambda x: x[x.CAGE_score > 0].length)
tool_gtfs['CAGE support relative'] = tool_gtfs['CAGE support absolute'] / tool_gtfs['num_transcripts']
tool_gtfs['TSS ratio absolute'] = tool_gtfs['CAGE_and_TSS'].apply(lambda x: x[x.TSS_ratio > 1.5].length)
tool_gtfs['TSS ratio relative'] = tool_gtfs['TSS ratio absolute'] / tool_gtfs['num_transcripts']

output = tool_gtfs.loc[:, ['tool', 'tissue', 'num_transcripts', 'CAGE support absolute', 'CAGE support relative', 'TSS ratio absolute', 'TSS ratio relative']]
output

,tool,tissue,num_transcripts,CAGE support absolute,CAGE support relative,TSS ratio absolute,TSS ratio relative
0,flair,aorta,86590,25377,0.293071,40896,0.472295
1,flair,brain,107799,31745,0.294483,47542,0.441024
2,flair,colon,40834,22250,0.544889,23491,0.575280
3,flair,heart,113023,31154,0.275643,82224,0.727498
4,flair,lung,93837,31754,0.338395,53391,0.568976
5,flair,muscle,25858,17584,0.680022,16323,0.631255
6,isotools,aorta,18326,11557,0.630634,10582,0.577431
7,isotools,brain,24205,17787,0.734848,12214,0.504606
8,isotools,colon,15890,12688,0.798490,10242,0.644556
9,isotools,heart,41235,23789,0.576913,24775,0.600825


# Export

In [34]:
import pickle

!mkdir -p /project/hfa_work/ENCODE/code/snakemake-pipeline/results/cage/
with open('/project/hfa_work/ENCODE/code/snakemake-pipeline/results/cage/support.pkl', 'wb') as f:
    pickle.dump(tool_gtfs, f)
output.to_csv('/project/hfa_work/ENCODE/code/snakemake-pipeline/results/cage/support.tsv', sep='\t', index=False)